<a href="https://colab.research.google.com/github/andrewdk1123/KoSentiment/blob/main/LSTM_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io

In [15]:
from google.colab import files

uploaded = files.upload()

Saving processed_training.csv to processed_training (1).csv


In [16]:
print(uploaded.keys())

dict_keys(['processed_training (1).csv'])


In [17]:
train_data = pd.read_csv(io.BytesIO(uploaded['processed_training (1).csv']), sep = ',')

cols_to_keep = ['label', 'sentence', 'tokenized_sentence', 'cleaned_tokens']
train_data = train_data.loc[:, cols_to_keep]
train_data.head()

,label,sentence,tokenized_sentence,cleaned_tokens
0,0,이제 슬슬 자리를 잡았다 싶었는데 아내가 세상을 떠났어.,"['이제', '슬슬', '자리를', '잡았다', '싶었는데', '아내가', '세상을...","['이제', '슬슬', '자리를', '잡았다', '싶었는데', '아내가', '세상을..."
1,0,내가 돈을 펑펑 쓰고 다닌다면서 남들이 뭐라고 하더라.,"['내가', '돈을', '펑', '##펑', '쓰고', '다닌', '##다면', '...","['내가', '돈을', '펑', '쓰고', '다닌', '남들이', '뭐라고', '하']"
2,0,아무 일도 아닌 것에 스트레스를 받는 나 자신이 너무 혐오스러워.,"['아무', '일도', '아닌', '것에', '스트레스를', '받는', '나', '...","['아무', '일도', '아닌', '것에', '스트레스를', '받는', '나', '..."
3,0,취업하기가 너무 힘들어.,"['취업', '##하기가', '너무', '힘들어', '.']","['취업', '너무', '힘들어']"
4,0,정년퇴직을 하고 나니 노후준비를 안 한 것이 후회가 돼.,"['정', '##년', '##퇴', '##직을', '하고', '나니', '노후', ...","['정', '하고', '나니', '노후', '안', '한', '것이', '후회', ..."


In [18]:
train_data.shape

(24504, 4)

In [19]:
uploaded = files.upload()

Saving processed_test.csv to processed_test (1).csv


In [20]:
print(uploaded.keys())

dict_keys(['processed_test (1).csv'])


In [21]:
test_data = pd.read_csv(io.BytesIO(uploaded['processed_test (1).csv']), sep = ',')

test_data = test_data.loc[:, cols_to_keep]
test_data.head()

,label,sentence,tokenized_sentence,cleaned_tokens
0,0,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,"['이번', '프로젝트', '##에서', '발표를', '하는데', '내가', '실수...","['이번', '프로젝트', '발표를', '하는데', '내가', '실수', '바람에'..."
1,0,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,"['회사에서', '중요한', '프로젝트를', '혼자', '하게', '됐는데', '솔...","['회사에서', '중요한', '프로젝트를', '혼자', '하게', '됐는데', '솔..."
2,0,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,"['상', '##사가', '너무', '무섭게', '생겨서', '친', '##해지는'...","['상', '너무', '무섭게', '생겨서', '친', '게', '너무', '두려워']"
3,0,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,"['이번에', '힘들게', '들어간', '첫', '직장', '##이거', '##든'...","['이번에', '힘들게', '들어간', '첫', '직장', '첫', '직장', '그..."
4,0,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,"['직장', '##에서', '동료', '##들이랑', '관계가', '안', '좋아질...","['직장', '동료', '관계가', '안', '좋아질', '봐', '걱정']"


In [22]:
test_data.shape

(6640, 4)

# Data Preparation

In [23]:
import ast
import torch
from torch.nn.utils.rnn import pad_sequence
from collections import Counter

# Convert the string representation of the list to an actual list
train_data['cleaned_tokens'] = train_data['cleaned_tokens'].apply(ast.literal_eval)
test_data['cleaned_tokens'] = test_data['cleaned_tokens'].apply(ast.literal_eval)

# Flatten the list of tokens
all_tokens = train_data['cleaned_tokens'].explode()

# Build a vocabulary using Counter
vocab = Counter(all_tokens)

# Create word-to-index and index-to-word dictionaries
word_to_index = {word: index + 1 for index, (word, _) in enumerate(vocab.most_common())}
index_to_word = {index: word for word, index in word_to_index.items()}

# Add a special token for padding with index 0
word_to_index[''] = 0
index_to_word[0] = ''

# Add a special token for out-of-vocabulary words with index len(vocab) + 1
word_to_index[''] = len(vocab) + 1
index_to_word[len(vocab) + 1] = ''

# Encode tokens using the word-to-index dictionary, replace out-of-vocabulary words with ''
train_data['encoded_tokens'] = train_data['cleaned_tokens'].apply(lambda tokens: [word_to_index.get(token, word_to_index['']) for token in tokens])

# Pad sequences to a specific length (adjust maxlen as needed)
maxlen = train_data['encoded_tokens'].apply(len).max()
padded_tokens = pad_sequence([torch.tensor(tokens) for tokens in train_data['encoded_tokens']], batch_first=True, padding_value=word_to_index['']).tolist()
train_data['padded_tokens'] = [list(tokens) for tokens in padded_tokens]

In [24]:
train_data.iloc[:,-2:].head()

,encoded_tokens,padded_tokens
0,"[17, 2627, 1306, 1695, 1033, 108, 842, 1456]","[17, 2627, 1306, 1695, 1033, 108, 842, 1456, 1..."
1,"[1, 92, 3587, 1396, 1740, 3218, 444, 249]","[1, 92, 3587, 1396, 1740, 3218, 444, 249, 1137..."
2,"[316, 690, 565, 929, 1019, 622, 8, 227, 3, 1962]","[316, 690, 565, 929, 1019, 622, 8, 227, 3, 196..."
3,"[79, 3, 82]","[79, 3, 82, 11373, 11373, 11373, 11373, 11373,..."
4,"[433, 28, 735, 145, 12, 25, 121, 236, 102]","[433, 28, 735, 145, 12, 25, 121, 236, 102, 113..."


In [25]:
print("Length of the Vocabulary is: ", len(vocab))

Length of the Vocabulary is:  11372


In [26]:
train_data['padded_tokens'].apply(len).unique()

array([38])

In [27]:
# Example: Reconstruct tokens for the first row
example_row_index = 0
reconstructed_tokens = [index_to_word[index] for index in train_data.loc[example_row_index, 'padded_tokens'] if index != 0]

# Display the reconstructed tokens
print(reconstructed_tokens)

['이제', '슬슬', '자리를', '잡았다', '싶었는데', '아내가', '세상을', '떠났', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [29]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [30]:
class BucketIterator:
    def __init__(self, data, batch_size, shuffle=True, seed=1123):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.seed = seed
        self.buckets = self.create_buckets()

    def create_buckets(self):
        buckets = {}
        for X, y in self.data:
            length = len(X)
            if length not in buckets:
                buckets[length] = []
            buckets[length].append((X, y))
        return buckets

    def shuffle_buckets(self):
        random.seed(self.seed)
        for key in self.buckets:
            random.shuffle(self.buckets[key])

    def __iter__(self):
        if self.shuffle:
            self.shuffle_buckets()

        for key in self.buckets:
            bucket = self.buckets[key]
            if not bucket:  # Check if the bucket is empty
                continue
            for i in range(0, len(bucket), self.batch_size):
                batch = bucket[i:i + self.batch_size]
                X_batch, y_batch = zip(*batch)

                X_batch = [torch.LongTensor(seq) for seq in X_batch]
                y_batch = torch.Tensor(y_batch).view(-1, 1)  # Ensure y_batch has shape (batch_size, 1)

                yield torch.stack(X_batch, dim=0), y_batch

In [31]:
train_X, val_X, train_y, val_y = train_test_split(train_data['padded_tokens'], train_data['label'], test_size=0.2, random_state=1123)

# Convert data and labels to numpy arrays and then PyTorch tensors
train_X = [torch.LongTensor(seq) for seq in train_X.values.tolist()]
train_y = torch.tensor(train_y.values.tolist())
val_X = [torch.LongTensor(seq) for seq in val_X.values.tolist()]
val_y = torch.tensor(val_y.values.tolist())

train_dataset = TensorDataset(torch.stack(train_X), train_y)
val_dataset = TensorDataset(torch.stack(val_X), val_y)

In [41]:
# Define a BucketIterator for training and validation
batch_size = 64
train_iterator = BucketIterator(train_dataset, batch_size=batch_size, shuffle=True)
val_iterator = BucketIterator(val_dataset, batch_size=batch_size, shuffle=False)

In [48]:
train_iterator.data[0]

(tensor([    6,    17,  1889,    17,   301,    27,  2648,  5823,    78, 11373,
         11373, 11373, 11373, 11373, 11373, 11373, 11373, 11373, 11373, 11373,
         11373, 11373, 11373, 11373, 11373, 11373, 11373, 11373, 11373, 11373,
         11373, 11373, 11373, 11373, 11373, 11373, 11373, 11373]),
 tensor(1))

In [32]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, pad_idx):

        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)

        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):

        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        return self.fc(hidden)

In [33]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = 0

model = RNN(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT,
            PAD_IDX)

model

RNN(
  (embedding): Embedding(11372, 100, padding_idx=0)
  (rnn): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [34]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,447,857 trainable parameters


In [35]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [36]:
def binary_accuracy(preds, y, threshold=0.5):
    # Apply threshold to the rounded predictions
    rounded_preds = (torch.sigmoid(preds) >= threshold).float()

    # Compare the rounded predictions to the ground truth
    correct = (rounded_preds == y).float()

    # Calculate accuracy
    acc = correct.sum() / len(correct)
    return acc

In [37]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        text, text_lengths = batch.text

        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [38]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text, text_lengths = batch.text

            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [39]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [42]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

AttributeError: ignored